In [ ]:
import os
import torch
import pandas as pd
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Amazon


#### globals
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.DataFrame(columns=['depth', "heads", "epochs", "activation_func", "final_train_score", "valid_score", "test_score"])

## tuning params
hyper_parameters = {
    "depth": list(range(1,6)),
    "heads": list(range(1, 9)),
    "epochs": list(range(50, 600, 50)),
    "activation_func": [F.elu, F.relu, F.tanh]
}

## load datset
dataset = 'Photo'
path = os.path.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())

## seeds
torch.manual_seed(0)

## add train / valid / test masks
dataset.shuffle()
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)

print(dataset.num_classes) # 8
# 8 * 20 --> 160 
data.train_mask[:160] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[160:660] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[660:1660] = True

data.to(device)

In [ ]:
from torch_geometric.nn import Sequential, GATConv, GCNConv
from torch.nn import ELU, Dropout, ReLU
from torch.nn import ModuleList

class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, config):
        super(Net, self).__init__()
        
        self.num_layers = config['depth']
        self.act = config['activation_func']
        self.dropout = 0.6
        
        self.convs = ModuleList()
        self.convs.append(
            GATConv(in_channels, 
                    8 if self.num_layers > 1 else out_channels, 
                    heads=config['heads'], 
                    dropout=self.dropout
            )
        )
        
        for i in range(1, self.num_layers):
            layer = None
            if i == self.num_layers - 1:
                # last layer
                layer = GATConv(config['heads'] * 8,  out_channels, heads=1, dropout=self.dropout)
            else:
                layer = GATConv(config['heads'] * 8,  8, heads=config['heads'], dropout=self.dropout)
                
            self.convs.append(layer)
        
    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if self.act is not None:
                x = self.act(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            
        return F.log_softmax(x, dim= -1)

In [ ]:
def log_stats(param_dict, train_acc, valid_acc, test_acc, filename):
    global df 
    
    df = df.append({
                'depth': param_dict['depth'], 
                "heads": param_dict['heads'], 
                "epochs": param_dict['epochs'], 
                "activation_func": param_dict['activation_func'], 
                "final_train_score": train_acc, 
                "valid_score": valid_acc, 
                "test_score": test_acc
    }, ignore_index=True)
    df.to_csv("./grid_search_" + filename + ".csv")
    

In [ ]:
from itertools import product
import numpy as np 

def grid_search(parameters, filename):
    # Convertion from dict to product of dicts
    
    keys, values = zip(*parameters.items())
    for prod in product(*values):
        param_dict = dict(zip(keys, prod))
        print(param_dict)
        model = Net(
            dataset.num_features,
            dataset.num_classes,
            param_dict
        ).to(device)

        optim = torch.optim.Adam(
            model.parameters(), 
            lr=0.01, 
            weight_decay=0
        )
        
        avg = np.empty([1,])
        max_val = 0
        early_stop_cnt = 0
        train_acc = val_acc = test_acc = 0
        for epoch in range(param_dict['epochs']):
            # train
            model.train()
            optim.zero_grad()
            out = model(data.x, data.edge_index)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optim.step()
            
            # valid
            accs = []
            with torch.no_grad():
                model.eval()
                out = model(data.x, data.edge_index)
                for _, mask in data('train_mask', 'val_mask', 'test_mask'):
                    acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
                    accs.append(acc)
            
            
            train_acc, val_acc, test_acc = accs
            
        print(train_acc, val_acc, test_acc)
        log_stats(param_dict, train_acc, val_acc, test_acc, filename)
        
        
    



In [ ]:
## grid search for amazon photos
grid_search(hyper_parameters, "photos")

In [ ]:
## load datset
dataset = 'Computers'
path = os.path.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())

## seeds
torch.manual_seed(0)

## add train / valid / test masks
dataset.shuffle()
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)

print(dataset.num_classes) # 10
# 10 * 20 --> 200
data.train_mask[:200] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[200:700] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[700:1700] = True

data.to(device)

In [ ]:

grid_search(hyper_parameters, "computers")